In [274]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import scale
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [275]:
# load data
data = pd.read_csv('./born_data.csv')

# separate x and y
y_label = '12D Epilepsy identification'
y_df = data[y_label].copy()
y_df[y_df <> 5] = 0
y_df[y_df == 5] = 1
x_df = data.drop(y_label, axis = 1).copy()

In [276]:
for col in x_df.columns:
    if '11 years' in col or '16 years' in col:
        x_df.drop(col, axis = 1, inplace = True)

In [277]:
# if more than 15 unique values, then treated as numerical
master_list = []

for col in x_df.columns:
    # get unique count
    unique_count = len(x_df[col].unique())
    
    # numerical variable
    if unique_count > 15:
        # get a copy for processing
        curr_df = pd.DataFrame(x_df[col])
        # handle NaN
        curr_df[curr_df.isnull()] = -1
        # standardize
        curr_df = pd.DataFrame(scale(curr_df.values))
        # reset index
        curr_df.index = x_df.index
        # change column label
        curr_df.columns = [col]
        # append to master list
        master_list.append(curr_df)
    # categorical variable
    else:
        # get a copy
        curr_df = x_df[col].copy()
        # get dummy
        # if contains NaN
        if curr_df.isnull().values.any():
            dummy = pd.get_dummies(curr_df, dummy_na = True)
        # if no NaN
        else:
            dummy = pd.get_dummies(curr_df, dummy_na = False)
        # reset columns
        dummy.columns = [str(col)+'='+str(value) for value in dummy.columns]
        # reset index
        dummy.index = x_df.index
        # append to master list
        master_list.append(dummy)
        
# concatenate master list
x_df_extended = pd.concat(master_list, axis = 1)

print 'data shape before dummification/standardization', x_df.shape
print 'data shape after dummification/standardization', x_df_extended.shape

data shape before dummification/standardization (18558, 64)
data shape after dummification/standardization (18558, 523)


In [278]:
def scorer(estimator, x, y):
    y_predict = estimator.predict(x)
    cm = confusion_matrix(y, y_predict)
    accuracy_all = estimator.score(x, y)
    accuracy_healthy = float(cm[0,0])/float(cm[0,0]+cm[0,1])
    accuracy_epilepsy = float(cm[1,1])/float(cm[1,0]+cm[1,1])
    return accuracy_all * accuracy_healthy * accuracy_epilepsy

In [279]:
weight_list = [{1:value} for value in 10**np.arange(2, 12, 2)]
parameters = {'class_weight':weight_list}
gscv = GridSearchCV(LogisticRegressionCV(), parameters, scoring = 'precision_macro')
gscv.fit(x_df_extended.values, y_df.values)
best_lr_model = gscv.best_estimator_

In [280]:
# get coefficient
coef = best_lr_model.coef_.flatten()

most_pos_var_index = np.argsort(coef)[-21:][::-1]
most_neg_var_index = np.argsort(coef)[0:3]

print 'most positive variables'
for index in most_pos_var_index:
    print coef[index], x_df_extended.columns[index]

most positive variables
51.472630467 0 Socio-economic group mother's husband (GRO 1951)=11.0
49.2220779015 0  Mothers blood group=nan
42.0422451964 0  Delivery-Supervision Groups=1.0
38.565878639 0 Smoking prior to pregnancy=1.0
27.6229317109 0  Whether labour induced=4.0
27.4683443145 0 Parity=9.0
27.3891800318 0 Socio-economic group mother's husband (GRO 1951)=2.0
24.8215501398 0  Duration membranes ruptured-hours=7.0
24.1722542549 0 Socio-economic group mother's husband (GRO 1951)=5.0
24.1114142671 0 Abnormality during pregnancy=8.0
23.9541580104 Region at NCDS1 (1965) - 7 years=6.0
23.9045859287 0  Siblings alive when mum left school=5.0
22.5402091956 0 M Number of persons per room=4.0
22.0050928736 0 Mother's weight in stones,1958=6.0
21.9002429848 0 Birthweight-gestational age for sex=2.0
20.8652875024 0  Whether labour induced=6.0
20.541892089 0  Birth order-all mums siblings=4.0
20.4253993097 0 Mother's present marital status=2.0
19.559796726 0 Parity=7.0
19.1905472226 Region a

In [281]:
best_lr_model = gscv.best_estimator_
kf = KFold(n_splits = 3)
adf = pd.DataFrame({}, columns = ['accuracy_all', 'accuracy_healthy', 'accuracy_epilepsy'])
index = 0

for train_index, test_index in kf.split(x_df_extended.values):
    x_train = x_df_extended.values[train_index, :]
    x_test = x_df_extended.values[test_index, :]
    y_train = y_df.values[train_index]
    y_test = y_df.values[test_index]
    
    best_lr_model.fit(x_train, y_train)
    
    y_predict = best_lr_model.predict(x_test)
    cm = confusion_matrix(y_test, y_predict)
    accuracy_all = best_lr_model.score(x_test, y_test)
    accuracy_epilepsy = float(cm[1,1])/float(cm[1,0]+cm[1,1])
    accuracy_healthy = float(cm[0,0])/float(cm[0,0]+cm[0,1])
    
    adf.loc[index] = [accuracy_all, accuracy_healthy, accuracy_epilepsy]

print adf.mean()

accuracy_all         0.975267
accuracy_healthy     0.978268
accuracy_epilepsy    0.050000
dtype: float64


In [282]:
print gscv.best_params_
print gscv.best_score_

{'class_weight': {1: 100000000}}
0.504158045901
